In [2]:

cosmosEndpoint = 'https://mm.documents.azure.com:443/'
cosmosMasterKey = 'MyMasterKey'

StatementMeta(pool03, 0, 3, Finished, Available)

In [3]:

cosmosDatabaseName = "sampleDB"
cosmosContainerName = "sampleContainer"

cfg = {
  "spark.cosmos.accountEndpoint" : cosmosEndpoint,
  "spark.cosmos.accountKey" : cosmosMasterKey,
  "spark.cosmos.database" : cosmosDatabaseName,
  "spark.cosmos.container" : cosmosContainerName,
  "spark.cosmos.write.strategy": "ItemOverwrite",
  "spark.cosmos.write.bulk.enabled": "true",
}

StatementMeta(pool03, 0, 4, Finished, Available)

In [4]:
spark.conf.set("spark.sql.catalog.cosmosCatalog", "com.azure.cosmos.spark.CosmosCatalog")
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountEndpoint", cosmosEndpoint)
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountKey", cosmosMasterKey)

# create a cosmos database using catalog api
spark.sql("CREATE DATABASE IF NOT EXISTS cosmosCatalog.{};".format(cosmosDatabaseName))

# create a cosmos container using catalog api
spark.sql("CREATE TABLE IF NOT EXISTS cosmosCatalog.{}.{} using cosmos.oltp TBLPROPERTIES(partitionKeyPath = '/id', manualThroughput = '1100')".format(cosmosDatabaseName, cosmosContainerName))

StatementMeta(, , , Finished, )

LIVY_JOB_TIMED_OUT: Livy session has failed. Session state: Dead. Error code: LIVY_JOB_TIMED_OUT. [plugins.mm-harley-synapse-workspace.pool03.0 WorkspaceType:<Synapse> CCID:<d18f8632-492c-4c76-894a-15f94d7893c6>] [Monitoring] Livy Endpoint=[https://hubservice1.eastus.azuresynapse.net:8001/api/v1.0/publish/ea414447-dee8-476e-b8bd-4f7df964caa6]. Livy Id=[0]  Job failed during run time with state=[dead]. Source: Dependency.

In [22]:
df = spark.createDataFrame((("cat-alive", "Schrodinger cat", 2, True), ("cat-dead", "Schrodinger cat", 2, False)))\
    .toDF("id","name","age","isAlive")


StatementMeta(pool02, 2, 23, Finished, Available)

In [23]:
display(df)

StatementMeta(pool02, 2, 24, Finished, Available)

SynapseWidget(Synapse.DataFrame, 21b209de-9122-48c5-b0cd-b9fa8b5ea788)

In [24]:
df\
   .write\
   .format("cosmos.oltp")\
   .options(**cfg)\
   .mode("APPEND")\
   .save()

StatementMeta(pool02, 2, 25, Finished, Available)

In [18]:
from pyspark.sql.functions import col

df = spark.read.format("cosmos.oltp").options(**cfg)\
 .option("spark.cosmos.read.inferSchema.enabled", "true")\
 .load()

df.filter(col("isAlive") == True)\
 .show()

StatementMeta(pool02, 2, 19, Finished, Available)

+---------+---------------+---+-------+
|       id|           name|age|isAlive|
+---------+---------------+---+-------+
|cat-alive|Schrodinger cat|  2|   true|
+---------+---------------+---+-------+

In [25]:
import pyspark.sql.functions as f
df_to_update = df.withColumn('name', f.lit('newname'))

StatementMeta(pool02, 2, 26, Finished, Available)

In [26]:
df_to_update \
        .write \
        .format("cosmos.oltp") \
        .mode("Append") \
        .options(**cfg) \
        .save()

StatementMeta(pool02, 2, 27, Finished, Available)

In [27]:
df = spark.read.format("cosmos.oltp").options(**cfg)\
 .option("spark.cosmos.read.inferSchema.enabled", "true")\
 .load().show()



StatementMeta(pool02, 2, 28, Finished, Available)

+---------+-------+---+-------+
|       id|   name|age|isAlive|
+---------+-------+---+-------+
|cat-alive|newname|  2|   true|
| cat-dead|newname|  2|  false|
+---------+-------+---+-------+